In [55]:
import matplotlib.pyplot as plt

In [56]:
class Water:
    def __init__(self, volume=0.0, NH4=0.0, NO3=0.0, bzv=0.0, O2=0.0):
        self.volume = volume # m3
        self.NH4 = NH4  # mg/l
        self.NO3 = NO3  # mg/l
        self.bzv = bzv  # mg/l
        self.O2 = O2  # mg/l

        self.NH4_absoluut = (self.NH4 * (self.volume*1000))  # mg
        self.NO3_absoluut = (self.NO3 * (self.volume*1000))  # mg
        self.bzv_absoluut = (self.bzv * (self.volume*1000))  # mg
        self.O2_absoluut = (self.O2 * (self.volume*1000))  # mg

    def update_NH4(self, NH4):
        self.NH4 = NH4  # mg/l
        self.NH4_absoluut = (self.NH4 * (self.volume*1000))  # mg
        
    def update_NO3(self, NO3):
        self.NO3 = NO3  # mg/l
        self.NO3_absoluut = (self.NO3 * (self.volume*1000))  # mg

    def update_bzv(self, bzv):
        self.bzv = bzv  # mg/l
        self.bzv_absoluut = (self.bzv * (self.volume*1000))  # mg

    def update_O2(self, O2):
        self.O2 = O2  # mg/l
        self.O2_absoluut = (self.O2 * (self.volume*1000))  # mg
    
    def get_absoluut(self):
        '''
        returns
            volume 
                m3
            NH4
                mg
            NO3
                mg
            bzv
                mg
            O2
                mg
        '''
        return self.volume, self.NH4_absoluut, self.NO3_absoluut, self.bzv_absoluut, self.O2_absoluut
    
    def __str__(self):
        return "Volume = {:.1f}m3\tNH4 = {:.4f}mg/l\tNO3 = {:.4f}mg/l\tvaste_stof = {:.4f}mg/l".format(self.volume, self.NH4, self.NO3, self.bzv)

In [57]:
class Tank(Water):
    def __init__(self, volume=0.0, NH4=0.0, NO3=0.0, bzv=0.0, O2=0.0):
        super().__init__(volume=volume, NH4=NH4, NO3=NO3, bzv=bzv, O2=O2)

        self.volume_tank = volume

In [58]:
class AnoxischeTank(Tank):
    def __init__(self, volume=0.0, NH4=0.0, NO3=0.0, bzv=0.0, denitrificatiesnelheid=0.1):
        super().__init__(volume=volume, NH4=NH4, NO3=NO3, bzv=bzv)
        
        self.denitrificatiesnelheid = denitrificatiesnelheid  # mg/s
        # self.bzv_snelheid = self.denitrificatiesnelheid/0.7
        self.molverhouding_bzv_no3 = 5.16

    def step(self, water_influent, water_recirculatie, t=900):
        volume_influent, NH4_influent, NO3_influent, bzv_influent, _ = water_influent.get_absoluut()
        volume_recirculatie, NH4_recirculatie, NO3_recirculatie, bzv_recirculatie, _ = water_recirculatie.get_absoluut()

        volume_tank, NH4_tank, NO3_tank, bzv_tank, _ = self.get_absoluut()

        volume_tot = volume_influent + volume_recirculatie + volume_tank  # m3
        # print(NH4_influent)
        # print(NH4_recirculatie)
        print(NH4_tank)
        NH4_tot = NH4_influent + NH4_recirculatie + NH4_tank  # mg
        NO3_tot = NO3_influent + NO3_recirculatie + NO3_tank  # mg
        bzv_tot = bzv_influent + bzv_recirculatie + bzv_tank  # mg
        
        # denitrificatie
        #2NO3 + 4H + bzv(2c) --> N2 + 2H2O + 2CO2
        # print("den*t: {:.2f}, no3_tot: {:.2f}, bzvtot/mv: {:.2f}".format(self.denitrificatiesnelheid * t, NO3_tot, bzv_tot/self.molverhouding_bzv_no3))
        NO3_verlies = min(self.denitrificatiesnelheid * t, NO3_tot, bzv_tot/self.molverhouding_bzv_no3)  # mg
        bzv_verlies = self.molverhouding_bzv_no3 * NO3_verlies  # mg

        NO3_tot = NO3_tot - NO3_verlies  # mg
        bzv_tot = bzv_tot - bzv_verlies  # mg

        volume_effluent = volume_tot - self.volume_tank   # m3
        NH4_effluent = (NH4_tot / (volume_tot * 1000))  # mg/l
        NO3_effluent = (NO3_tot / (volume_tot * 1000))  # mg/l
        bzv_effluent = (bzv_tot / (volume_tot * 1000))  # mg/l

        water_effluent = Water(volume=volume_effluent, NH4=NH4_effluent, NO3=NO3_effluent, bzv=bzv_effluent)

        self.update_NH4(NH4_tot / (volume_tot * 1000))  # mg/l
        self.update_NO3(NO3_tot / (volume_tot * 1000))  # mg/l
        self.update_bzv(bzv_tot / (volume_tot * 1000))  # mg/l


        return water_effluent

In [59]:
class Beluchtingstank(Tank):
    def __init__(self, volume=0.0, NH4=0.0, NO3=0.0, vaste_stof=0.0, O2=0.0, temperatuur=0.0):
        super().__init__(volume, NH4, NO3, vaste_stof)
        self.O2 = O2  # mg/l
        self.temperatuur = temperatuur  # °C
        

In [60]:
influent = Water(volume=25, NH4=5, bzv=0.1)

anoxische_tank = AnoxischeTank(volume=1000)
beluchtingstank = Beluchtingstank(volume=1000)
# nabezinktank = Nabezinktank(volume=1000)

recirculatie = Water()
effluent = Water()

In [61]:
nh4_range = list()

time_range = list(range(10))

for t in time_range:
    beluchtingstank_stroom = anoxische_tank.step(influent, recirculatie)
    # recirculatie, effluent = beluchtingstank.step(beluchtingstank_stroom)
    # effluent = nabezinktank.step(nabezink_stroom)
    volume, NH4, NO3, bzv, O2 = beluchtingstank_stroom.get_absoluut()
    # print(influent.bzv)
    # print(beluchtingstank_stroom.bzv)

125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
125000
0.0
0.0
